In [381]:
# Import packages
import numpy as np
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

#Loading the data
c = np.loadtxt('NewMacroSeries.txt', delimiter=",",usecols=0)
k = np.loadtxt('NewMacroSeries.txt', delimiter=",",usecols=1)
w = np.loadtxt('NewMacroSeries.txt', delimiter=",",usecols=2)
r = np.loadtxt('NewMacroSeries.txt', delimiter=",",usecols=3)
y = np.loadtxt('NewMacroSeries.txt', delimiter=",",usecols=4)
data=np.vstack([c,k,w,r,y])

In [382]:
# Define function that draws N x S test score values from a normal distribution
def norm_draws(unif_vals, mu, sigma):
    norm_draws = sts.norm.ppf(unif_vals, loc=mu, scale=sigma)
    return norm_draws

In [383]:
# Define function that generates data moments.
def data_moments(xvals):
    mean_c = np.mean(xvals[0])
    mean_k = np.mean(xvals[1])
    mean_cy = np.mean(np.divide(xvals[0],xvals[4]))
    var_y = np.var(xvals[4])
    corr_cc = np.corrcoef(xvals[0][0:-2],xvals[0][1:-1])[0,1]
    corr_ck = np.corrcoef(xvals[0][0:-1],xvals[4][0:-1])[0,1]
    return mean_c, mean_k, mean_cy, var_y, corr_cc, corr_ck

In [384]:
#Define Data Simulation Function.
def sim_data(params,args):
    mu, rho, sigma,alpha=params
    xvals, k_1,beta, T, S, W_hat=args  
    unif_vals_1 = sts.uniform.rvs(0, 1, size=(T+1,S)) #Uniform Draws
    eps=norm_draws(unif_vals_1[1],0,sigma)         #Error Terms

    #Generating z_t
    z_t=np.linspace(0,T,T+2)
    for i in range(0,T+2):
        if i==0:
              z_t[i]=mu
        else:
              z_t[i]=z_t[i-1]*rho+mu*(1-rho)+eps[i-1]
    
    #Generating k_t
    k_t=np.linspace(0,T,T+1)
    for i in range(0,T):
        if i==0:
            k_t[i]=k_1
        else:
            k_t[i]=np.log(alpha)+np.log(beta)+z_t[i+1]+alpha*np.log(k_t[i-1])

    #Generating w_t and r_t
    z_t = z_t[1:T+2]
    w_t = np.log(1-alpha) + z_t + np.log(k_t)*alpha
    r_t = np.log(alpha) + z_t + np.log(k_t)*(1-alpha)
    w_t = w_t[0:T]
    r_t = r_t[0:T]
    z_t = z_t[0:T]
    c_t = r_t*k_t[0:T] + w_t - k_t[1:T+1]
    y_t = np.exp(z_t)*k_t[0:T]**alpha

    sim_vals = [c_t,k_t,w_t,r_t,y_t]
    return sim_vals


In [385]:
#Define Data Simulation Function with S=loop.
def sim_data2(params,args):
    mu, rho, sigma,alpha=params
    xvals, k_1,beta, T, S, W_hat=args  
    for s in range(0,S):
        unif_vals_1 = sts.uniform.rvs(0, 1, size=(T+1,S)) #Uniform Draws
        eps=norm_draws(unif_vals_1[1],0,sigma)         #Error Terms

    #Generating z_t
        z_t=np.linspace(0,T,T+2)
        for i in range(0,T+2):
            if i==0:
                  z_t[i]=mu
            else:
                  z_t[i]=z_t[i-1]*rho+mu*(1-rho)+eps[i-1]
    
    #Generating k_t
        k_t=np.linspace(0,T,T+1)
        for i in range(0,T):
            if i==0:
                k_t[i]=k_1
            else:
                k_t[i]=np.log(alpha)+np.log(beta)+z_t[i+1]+alpha*np.log(k_t[i-1])

    #Generating w_t and r_t
        z_t = z_t[1:T+2]
        w_t = np.log(1-alpha) + z_t + np.log(k_t)*alpha
        r_t = np.log(alpha) + z_t + np.log(k_t)*(1-alpha)
        w_t = w_t[0:T]
        r_t = r_t[0:T]
        z_t = z_t[0:T]
        c_t = r_t*k_t[0:T] + w_t - k_t[1:T+1]
        y_t = np.exp(z_t)*k_t[0:T]**alpha
        sim_vals_t = [c_t,k_t,w_t,r_t,y_t]
        if s==0:
            sim_vals_s = sim_vals_t
        else:
            sim_vals_s = sim_vals_s + np.divide((c_t,k_t,w_t,r_t,y_t),S)
    return sim_vals_s

In [386]:
#Define Moment Error Function
def err_vec(data_vals, sim_vals, simple):
    mean_c_d, mean_k_d, mean_cy_d, var_y_d, corr_cc_d, corr_ck_d = data_moments(data_vals)
    moms_data =[[mean_c_d], [mean_k_d],[mean_cy_d],[var_y_d],[corr_cc_d],[corr_ck_d]]
    mean_c_s, mean_k_s, mean_cy_s, var_y_s, corr_cc_s, corr_ck_s = data_moments(sim_vals)
    moms_model = np.array([[mean_c_s], [mean_k_s],[mean_cy_s],[var_y_s],[corr_cc_s],[corr_ck_s]])
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    return err_vec

#Define Criterion Functions
def criterion(params, *args):
    mu, rho, sigma, alpha = params
    xvals, k_1, beta, T, S, W_hat = args
    sim_vals=sim_data(params,args)               #Using sim_data
    err = err_vec(xvals, sim_vals, simple=False)
    crit_val = np.dot(np.dot(err.T, W_hat), err) 
    return crit_val

#Define Criterion Functions
def criterion2(params, *args):
    mu, rho, sigma, alpha = params
    xvals, k_1, beta, T, S, W_hat = args
    sim_vals=sim_data2(params,args)               #Using sim_data2
    err = err_vec(xvals, sim_vals, simple=False)
    crit_val = np.dot(np.dot(err.T, W_hat), err) 
    return crit_val

In [433]:
#Initial Values
mu = 9.1
rho = 0.01
sigma = 0.03
alpha = 0.40
params=(mu, rho, sigma, alpha)

#Assumptions
xvals=data
k_1=np.mean(xvals[1])
beta=0.99
S=1000
T=100
nm=6            #number of moment
W_hat=np.eye(nm)
args=(xvals, k_1, beta, T, S, W_hat)

#Bounds
mub=(5,14)
rhob=(-0.99,0.99)
sigmab=(0.01,1.1)
alphab=(0.01,0.99)

#crit_test = criterion(params, *args)
crit_test = criterion2(params, *args)  #S=1000
sim_vals=sim_data(params, args)
mean_c_d, mean_k_d, mean_cy_d, var_y_d, corr_cc_d, corr_ck_d = data_moments(data)
mean_c_s, mean_k_s, mean_cy_s, var_y_s, corr_cc_s, corr_ck_s = data_moments(sim_vals)
print('Data Mean of Consumption =', mean_c_d,'Data Mean of Capital =',mean_k_d)
print('Model Mean of Consumption =', mean_c_s,'Model Mean of Capital =',mean_k_s)
print('')
print('Data Mean of Consump/GDP =', mean_cy_d,'Data Variance of GDP =',var_y_d)
print('Model Mean of Consump/GDP =', mean_cy_s,'Model Variance of GDP =',var_y_s)
print('')
print('Data AR of Consumption =', corr_cc_d, 'Data Corr of Consump and Capital =',corr_ck_d )
print('Model AR of Consumption =', corr_cc_s, 'Model Corr of Consump and Capital =',corr_ck_s )
print('')
print('Criterion Value =',crit_test)
print('')
results = opt.minimize(criterion, params, args=args, method='L-BFGS-B',bounds=(mub,rhob,sigmab,alphab,))
print('SMM Estimates')
mu_SMM, rho_SMM, sigma_SMM, alpha_SMM = results.x
print('mu_SMM=', mu_SMM,'rho_SMM=', rho_SMM,'sigma_SMM=', sigma_SMM, 'alpha_SMM=', alpha_SMM)
print('')
print(results)

Data Mean of Consumption = 9281790.48567 Data Mean of Capital = 6643985.1383
Model Mean of Consumption = 1171623.11913 Model Mean of Capital = 65791.9583206

Data Mean of Consump/GDP = 0.5842 Data Variance of GDP = 2.83778250589e+13
Model Mean of Consump/GDP = 0.24234752633 Model Variance of GDP = 236897118563.0

Data AR of Consumption = 0.94085870125 Data Corr of Consump and Capital = 1.0
Model AR of Consumption = 0.992268566519 Model Corr of Consump and Capital = 0.999998588701

Criterion Value = [[ 2.80125583]]

SMM Estimates
mu_SMM= 9.10000646065 rho_SMM= 0.0100013164036 sigma_SMM= 0.0300013714618 alpha_SMM= 0.40000075672

      fun: array([[ 3.07409021]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -600902.5059293 ,   107654.31858224,  -106478.93319771,
       -4478728.65397252])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 100
      nit: 3
   status: 0
  success: True
        x: array([ 9.10000646,  0.01000132,  0.0300013

In [434]:
#SMM Results
params_SMM=(mu_SMM, rho_SMM, sigma_SMM, alpha_SMM)
draw_SMM=sim_data(params_SMM, args)
mean_c_s, mean_k_s, mean_cy_s, var_y_s, corr_cc_s, corr_ck_s = data_moments(draw_SMM)
err_1 = err_vec(data, draw_SMM,False).reshape(6,)
print('Data Mean of Consumption =', mean_c_d,'Data Mean of Capital =',mean_k_d)
print('Model Mean of Consumption =', mean_c_s,'Model Mean of Capital =',mean_k_s)
print('')
print('Data Mean of Consump/GDP =', mean_cy_d,'Data Variance of GDP =',var_y_d)
print('Model Mean of Consump/GDP =', mean_cy_s,'Model Variance of GDP =',var_y_s)
print('')
print('Data AR of Consumption =', corr_cc_d, 'Data Corr of Consump and Capital =',corr_ck_d )
print('Model AR of Consumption =', corr_cc_s, 'Model Corr of Consump and Capital =',corr_ck_s )
print('')
print('Criterion Value =',crit_test[0])
print('')
print('Error vector 1 =', err_1)


Data Mean of Consumption = 9281790.48567 Data Mean of Capital = 6643985.1383
Model Mean of Consumption = 1168918.4417 Model Mean of Capital = 65791.9576604

Data Mean of Consump/GDP = 0.5842 Data Variance of GDP = 2.83778250589e+13
Model Mean of Consump/GDP = 0.251675602939 Model Variance of GDP = 218303903718.0

Data AR of Consumption = 0.94085870125 Data Corr of Consump and Capital = 1.0
Model AR of Consumption = 0.99276251107 Model Corr of Consump and Capital = 0.999998040451

Criterion Value = [ 2.80125583]

Error vector 1 = [ -8.74063259e-01  -9.90097516e-01  -5.69196161e-01  -9.92307236e-01
   5.51664238e-02  -1.95954897e-06]


In [ ]:
# Criterion function 1 runs only single loop, and criterion function 2 runs loops for S times.
# With similiar initial values, criterion function 2 will have less criterion value than criterion function 1.